Notebook imitant:
    https://towardsdatascience.com/causal-machine-learning-for-econometrics-causal-forests-5ab3aec825a7

In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 

In [85]:
#Read main data to DataFrame
df = pd.read_stata('Cash_and_Childhood_Development_Replication/macoursetal_main.dta')  

In [86]:
# "hogarid_old","cp_old","unique_05", "cpmom_06" seran usats com a index
df_out=df[["a5sscore_i_06","a6smemory_p1_06","a6smemory_p2_06","a7a_delay_06","a7b_delay_06",
        "a7c_delay_06","a7d_delay_06","a9sgrossmotor_06","height_06","weight_06","z_tvip_06","z_social_06",
        "z_language_06","z_finmotor_06","z_memory_06","z_grmotor_06","z_legmotor_06","z_behavior_06","z_height_06",
        "z_weight_06","z_all_06"]]


In [87]:
df_out.isna().sum().sort_values()

z_all_06            1185
a7a_delay_06        1200
z_social_06         1204
a7b_delay_06        1220
z_language_06       1224
a7c_delay_06        1242
z_finmotor_06       1246
a7d_delay_06        1254
z_grmotor_06        1258
z_height_06         1429
height_06           1429
weight_06           1429
z_weight_06         1429
a9sgrossmotor_06    2656
a6smemory_p1_06     2669
z_legmotor_06       2673
a5sscore_i_06       2679
z_memory_06         2684
z_tvip_06           2694
z_behavior_06       2891
a6smemory_p2_06     3141
dtype: int64

In [88]:
#Ens quedem amb els que tenen <1300 nans

df_out=df_out[df_out.isna().sum().sort_values()[0:9].index]

# Income variables

In [89]:
df1_extended=df[["hogarid_old","s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05",
        "s1hhsz_15_24_05","s1hhsz_25_64_05","s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05",
        "s3ap23_stime_h_05","s3ap24_htime_h_05","s3ap25_hqtime_h_05","s3atoilet_hh_05","s3awater_access_hh_05",
        "s3aelectric_hh_05","s4p6_vitamina_i_05","s4p7_parasite_i_05","s11ownland_hh_05","cons_food_pc_05","cons_tot_pc_05",
        "tvip_05","height_05","a10whz_05","weight_05","itt_i","itt_all_i","yrsedfath","age_transfer","bweight",
        "s4p7_parasite_i_06","T","male","TREAT1","TREAT2","TREAT3","TREAT4","ed_mom","MUN1","MUN2","MUN3","MUN4",
        "MUN5","MUN6","com_haz_05","com_waz_05","com_tvip_05","com_control_05","com_vit_05","com_deworm_05","com_notvip",
        "sample06","vitamiron_06",#"weighted_05",
        "propfood_05","prstap_f_05","pranimalprot_f_05","prfruitveg_f_05"]]

In [90]:
df1=df[["hogarid_old","s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05",
        "s1hhsz_15_24_05","s1hhsz_25_64_05","s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05",
        "s3ap23_stime_h_05","s3ap24_htime_h_05","s3ap25_hqtime_h_05","s3atoilet_hh_05","s3awater_access_hh_05",
        "s3aelectric_hh_05","s4p6_vitamina_i_05","s4p7_parasite_i_05","s11ownland_hh_05","cons_food_pc_05","cons_tot_pc_05",
        "height_05","a10whz_05","weight_05","yrsedfath","age_transfer","bweight",
        "s4p7_parasite_i_06","T","male","ed_mom","MUN1","MUN2","MUN3","MUN4",
        "MUN5","MUN6","com_haz_05","com_waz_05","com_vit_05","com_deworm_05",
        "vitamiron_06", "propfood_05","prstap_f_05","pranimalprot_f_05","prfruitveg_f_05"]]

In [91]:
df1.rename(columns = {'T':'tr'}, inplace = True)
df1_extended.rename(columns = {'T':'tr'}, inplace = True)

C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\3261090467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns = {'T':'tr'}, inplace = True)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\3261090467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_extended.rename(columns = {'T':'tr'}, inplace = True)


In [92]:
#fem una copia del df
def copy_df(df):
   return df.copy()

In [93]:
#eliminem individus amb >delta*100% columnes buides
def drop_ind_missing(df, delta):
    thresh = len(df.columns)*delta
    df.dropna(axis=0, thresh=thresh, inplace=True)
    return df

In [94]:
#eliminem individus amb age transfer<age
def drop_out_missing(df, age=-11):
    nan_rows=df[df.age_transfer<age].index
    df.drop(nan_rows, inplace=True)
    return df

In [95]:
#categorical to ordinal
def ordinal(df, cols):
    enc = OrdinalEncoder()
    enc.fit(df[cols])
    df[cols] = enc.transform(df[cols])
    
    return df

In [96]:
#imputem NaNs

#imputem mitjanes
def mean_imputer(df, cols):
    for col in cols:
        df[col].fillna(df[col].mean(), inplace=True)
    return df

#imputem el valor més frequent de la columna dintre el hh
def hh_mf_imputer(df,cols):
    index=df.index
    for col in cols:
        col_hh=[df[df.hogarid_old==familia][[col,"age_transfer"]].sort_values(by=['age_transfer'])[col]
                .values for familia in df.hogarid_old]
        for i in range(len(index)):
               if np.isnan(df[col][index[i]]):
                    non_nan_len=np.count_nonzero(~np.isnan(col_hh[i]))
                    if(non_nan_len!=0):
                        vals,counts = np.unique(col_hh[i], return_counts=True)
                        ind = np.argmax(counts)
                        df.loc[index[i],col]=vals[ind]
    return df             
                            
                        

In [97]:
#eliminem outliers

def outlier_removal(df, cols):
    outlier_threshold = []
    for col in cols:
            q3 = np.nanpercentile(df[col], 75)
            q1 = np.nanpercentile(df[col], 25)
            iqr = q3 - q1
            out_low = q1 - 3*iqr
            out_high = q3 + 3*iqr
            outlier_threshold.append([out_low, out_high])
    outliers_indexs=[]
    for col,k in zip(cols,range(len(cols))):
        #outliers_indexs_in_col=[]
        for i in df[col].index:
            if df[col][i]<outlier_threshold[k][0] or df[col][i]>outlier_threshold[k][1]:
                outliers_indexs.append(i)
        #outliers_indexs.append(outliers_indexs_in_col)
    return df.drop(list(set(outliers_indexs)), axis=0)

In [98]:
def normalize(df, cols):
    Nscaler = MinMaxScaler()

    Nscaler.fit(df[cols])
    df[cols] = Nscaler.transform(df[cols])
    
    return df

def standardize(df, cols):
    Sscaler = StandardScaler()

    Sscaler.fit(df[cols])
    df[cols] = Sscaler.transform(df[cols])
    
    return df

In [99]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

mean_cols=["bweight", "height_05", "a10whz_05", "weight_05"]
hh_mf_cols=["s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05","s1hhsz_15_24_05","s1hhsz_25_64_05",
            "s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05","s3atoilet_hh_05",
            "s3awater_access_hh_05","s3aelectric_hh_05","s11ownland_hh_05","s4p7_parasite_i_06","ed_mom","vitamiron_06",
           "s4p6_vitamina_i_05", "s4p7_parasite_i_05", "cons_food_pc_05", "yrsedfath", "propfood_05", "prstap_f_05"
            ,  "pranimalprot_f_05",  "prfruitveg_f_05"]

categorical_cols=df1.select_dtypes(exclude=["number","bool_","object_"]).columns
outlier_cols=["pranimalprot_f_05", "bweight", "prfruitveg_f_05"]

normal_cols =["height_05","a10whz_05","weight_05","com_haz_05","com_waz_05"]

Nnormal_cols = ['s1age_head_05', 's3ap23_stime_h_05', 's3ap24_htime_h_05', 's3ap25_hqtime_h_05', 'cons_food_pc_05', 
                'cons_tot_pc_05', 'tvip_05', 'yrsedfath','age_transfer', 'bweight', 'ed_mom', 'com_tvip_05', 'com_control_05']


cleaned_df=(df1.pipe(copy_df)
            .pipe(drop_ind_missing, 0.8)
            .pipe(drop_out_missing)
            .pipe(ordinal, categorical_cols)
            .pipe(hh_mf_imputer, hh_mf_cols)
            .pipe(mean_imputer, mean_cols+hh_mf_cols)
            .pipe(outlier_removal, outlier_cols) #outliers
            .pipe(standardize, normal_cols)) #estandaritzacio
            #.pipe(normalize, Nnormal_cols)) #normalitzacio
                         

In [100]:
mean_cols=["bweight", "tvip_05", "height_05", "a10whz_05", "weight_05"]
hh_mf_cols=["s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05","s1hhsz_15_24_05","s1hhsz_25_64_05",
            "s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05","s3atoilet_hh_05",
            "s3awater_access_hh_05","s3aelectric_hh_05","s11ownland_hh_05","s4p7_parasite_i_06","ed_mom","vitamiron_06",
           "s4p6_vitamina_i_05", "s4p7_parasite_i_05", "cons_food_pc_05", "yrsedfath", "propfood_05", "prstap_f_05"
            ,  "pranimalprot_f_05",  "prfruitveg_f_05"]

categorical_cols=df1_extended.select_dtypes(exclude=["number","bool_","object_"]).columns
outlier_cols=["pranimalprot_f_05", "bweight", "prfruitveg_f_05"]

normal_cols =["height_05","a10whz_05","weight_05","com_haz_05","com_waz_05"]

Nnormal_cols = ['s1age_head_05', 's3ap23_stime_h_05', 's3ap24_htime_h_05', 's3ap25_hqtime_h_05', 'cons_food_pc_05', 
                'cons_tot_pc_05', 'tvip_05', 'yrsedfath','age_transfer', 'bweight', 'ed_mom', 'com_tvip_05', 'com_control_05']

cleaned_df_ext=(df1_extended.pipe(copy_df)
            .pipe(drop_ind_missing, 0.8)
            .pipe(drop_out_missing)
            .pipe(ordinal, categorical_cols)
            .pipe(hh_mf_imputer, hh_mf_cols)
            .pipe(mean_imputer, mean_cols+hh_mf_cols)
            .pipe(outlier_removal, outlier_cols) #outliers
            .pipe(standardize, normal_cols)) #estandaritzacio
            #.pipe(normalize, Nnormal_cols)) #normalitzacio

In [101]:
treatment = 'tr'
covariates = ["s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05",
        "s1hhsz_15_24_05","s1hhsz_25_64_05","s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05",
        "s3ap23_stime_h_05","s3ap24_htime_h_05","s3ap25_hqtime_h_05","s3atoilet_hh_05","s3awater_access_hh_05",
        "s3aelectric_hh_05","s4p6_vitamina_i_05","s4p7_parasite_i_05","s11ownland_hh_05","cons_food_pc_05","cons_tot_pc_05",
        "height_05","a10whz_05","weight_05","yrsedfath","age_transfer","bweight",
        "s4p7_parasite_i_06","male","ed_mom","MUN1","MUN2","MUN3","MUN4",
        "MUN5","MUN6","com_haz_05","com_waz_05","com_vit_05","com_deworm_05",
        "vitamiron_06", "propfood_05","prstap_f_05","pranimalprot_f_05","prfruitveg_f_05"]

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def ate (treatment, outcome, covariates, cleaned_df, df_out):
    temp_df=df_out.iloc[cleaned_df.index]
    df=cleaned_df.drop(cleaned_df[temp_df[outcome].isna()].index, axis=0)
    df_temp=df_out.iloc[df.index]
    df[outcome]=df_temp[outcome]
    df.drop("hogarid_old", axis=1, inplace=True)
    print(df.shape)
    
    # split data into train and test sets 
    train, test = train_test_split(df, test_size=0.2)
    
    y_train = train[outcome]
    X_train = train[covariates+["tr"]]

    X_test = test[covariates+["tr"]]
    y_test =  test[outcome]
    
    forest_reg = RandomForestRegressor(n_estimators=20, random_state=42)
    forest_reg.fit(X_train, y_train)

    counterfactual_df=df.drop(outcome,axis=1)
    a=counterfactual_df["tr"]+1
    counterfactual_df["tr"]=a%2
    counterfactual=forest_reg.predict(counterfactual_df)
    observed=df[outcome]
    counterfactuals=pd.Series(counterfactual, index=observed.index)

    ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
    ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)
    ite=pd.concat([ite_temp,ite_temp1])
    ite.sort_index(inplace=True)
    
    return ite

In [133]:
ate_vec=[ate(treatment, outcome, covariates, cleaned_df, df_out) for outcome in df_out.columns]

(3160, 47)


C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)


(3141, 47)


C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)


(3141, 47)


...

C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)


(3087, 47)


C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)


(3087, 47)


C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\38251217.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)


In [107]:
treatment = 'tr'
covariates_ext=["s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05",
        "s1hhsz_15_24_05","s1hhsz_25_64_05","s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05",
        "s3ap23_stime_h_05","s3ap24_htime_h_05","s3ap25_hqtime_h_05","s3atoilet_hh_05","s3awater_access_hh_05",
        "s3aelectric_hh_05","s4p6_vitamina_i_05","s4p7_parasite_i_05","s11ownland_hh_05","cons_food_pc_05","cons_tot_pc_05",
        "tvip_05","height_05","a10whz_05","weight_05","itt_i","itt_all_i","yrsedfath","age_transfer","bweight",
        "s4p7_parasite_i_06","male","TREAT1","TREAT2","TREAT3","TREAT4","ed_mom","MUN1","MUN2","MUN3","MUN4",
        "MUN5","MUN6","com_haz_05","com_waz_05","com_tvip_05","com_control_05","com_vit_05","com_deworm_05","com_notvip",
        "sample06","vitamiron_06",#"weighted_05",
        "propfood_05","prstap_f_05","pranimalprot_f_05","prfruitveg_f_05"]
ate_vec_ext=[ate(treatment, outcome, covariates_ext, cleaned_df_ext, df_out) for outcome in df_out.columns]

C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\397396693.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp=pd.Series([observed[i]-counterfactuals[i] for i in observed.index if cleaned_df.tr[i]==1], index=df[cleaned_df.tr==1].index)
C:\Users\Usuari\AppData\Local\Temp\ipykernel_15756\397396693.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ite_temp1=pd.Series([counterfactuals[i]-observed[i] for i in observed.index if cleaned_df.tr[i]==0], index=df[cleaned_df.tr==0].index)
C:\Users\Usuari\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit

In [123]:
df_ite=pd.DataFrame([np.mean(ite) for ite in ate_vec], index=df_out.columns, columns=["ATE_normal_cov"])

In [124]:
df_ite["ATE_extended_cov"]=[np.mean(ite) for ite in ate_vec_ext]

In [125]:
df_ite["ATE_paper"]=[0.08,"?",0.13,"?",0.139,"?", 0.038,"?",0.06]

In [126]:
df_ite

,ATE_normal_cov,ATE_extended_cov,ATE_paper
z_all_06,0.084013,0.085483,0.08
a7a_delay_06,-0.060188,-0.091429,?
z_social_06,0.314620,0.356739,0.13
a7b_delay_06,-0.006584,-0.010186,?
z_language_06,0.278611,0.249633,0.139
a7c_delay_06,-0.050532,-0.058362,?
z_finmotor_06,0.357017,0.424841,0.038
a7d_delay_06,-0.055442,-0.065971,?
z_grmotor_06,0.232994,0.279858,0.06


In [127]:
df1.shape

(4511, 47)

In [128]:
df1_extended.shape

(4511, 58)

In [131]:
cleaned_df.shape

(3451, 47)